# Subset selection for Hyperparameter tuning

In this tutorial, we will look at an example showing how to integrate GradMatchPB-Warm subset selection strategy in typical hyperparameter tuning loop
for configuration model training on CIFAR100 dataset with TPE as hyper-parameter search algorithm and ASHA as hyper-parameter scheduler.

### Cloning CORDS repository

In [2]:
!git clone https://github.com/decile-team/cords.git
%cd cords/
%ls

Cloning into 'cords'...
remote: Enumerating objects: 5140, done.
remote: Counting objects: 100% (455/455), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 5140 (delta 337), reused 327 (delta 256), pack-reused 4685
Receiving objects: 100% (5140/5140), 58.47 MiB | 23.96 MiB/s, done.
Resolving deltas: 100% (3188/3188), done.
/content/cords
benchmarks/   examples/      requirements/  train_ssl.py
CITATION.CFF  gradio_hpo.py  setup.py       transformers_train_sl.py
configs/      gradio_sl.py   tests/         tutorial/
cords/        LICENSE.txt    train_hpo.py
docs/         README.md      train_sl.py


### Install prerequisite libraries of CORDS

In [3]:
!pip install dotmap
!pip install apricot-select
!pip install ray[default]
!pip install ray[tune]
!pip install datasets
!pip install transformers
!pip install sentence-transformers
!pip install scikit-learn
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 KB 6.8 MB/s eta 0:00:00
  Created wheel for apricot-select: filename=apricot_select-0.6.1-py3-none-any.whl size=48786 sha256=f17c811a7acf0ac53fb6de4c23cf2e912de4388a9a4f541938b0fbea38e5ec04
  Stored in directory: /root/.cache/pip/wheels/31/9d/60/56b3035d46924261240d200e1b3e99094ad23f2223a6d58b49
Successfully built apricot-select
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 4.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 20.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 5

# Install Submodlib

In [4]:
%cd ..
!git clone https://github.com/decile-team/submodlib.git
%cd submodlib
!pip install .
%cd ../cords

/content
Cloning into 'submodlib'...
remote: Enumerating objects: 2563, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 2563 (delta 0), reused 0 (delta 0), pack-reused 2559
Receiving objects: 100% (2563/2563), 30.56 MiB | 28.19 MiB/s, done.
Resolving deltas: 100% (1909/1909), done.
/content/submodlib
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/submodlib
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 88.2 MB/s eta 0:00:00
  Created wheel for submodlib: filename=submodlib-1.1.5-cp38-cp38-linux_x86_64.whl size=523302 sha256=89c1b0efc049b7f2ce4aa7a55960be68697f366d3ab3fdecbf83237ba7dc4d2f
  Stored in directory: /tmp/pip-ephem-wheel-cache-nf9ibwqy/wheels/fd/7d/9b/2f261fe

In [33]:
!pip install ax-platform sqlalchemy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 KB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 KB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.7/730.7 KB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 KB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.0 MB/s eta 0:00:00


In [47]:
!pip install dragonfly-opt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 KB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for dragonfly-opt: filename=dragonfly_opt-0.1.7-cp38-cp38-linux_x86_64.whl size=419964 sha256=9373c11657f2445707104087e333dfb3ae891c017681905aa14658b220406af5
  Stored in directory: /root/.cache/pip/wheels/f0/c3/41/09455a3380a6e7f6d1d9e6a734e5314185a31fdea5669ac034
Successfully built dragonfly-opt


# Copying glove embeddings from drive

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
%mkdir /content/data/

mkdir: cannot create directory ‘/content/data/’: File exists


In [40]:
%mkdir /content/data/glove.6B

mkdir: cannot create directory ‘/content/data/glove.6B’: File exists


In [41]:
%cp /content/drive/MyDrive/glove.6B.300d.txt /content/data/glove.6B/
%cp -r /content/drive/MyDrive/data/* /content/data/

### Import necessary libraries

In [42]:
import argparse
from cords.utils.config_utils import load_config_data
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.suggest.bayesopt import BayesOptSearch
from ray.tune.suggest.skopt import SkOptSearch
from ray.tune.suggest.dragonfly import DragonflySearch
from ray.tune.suggest.ax import AxSearch
from ray.tune.suggest.bohb import TuneBOHB
from ray.tune.suggest.nevergrad import NevergradSearch
from ray.tune.suggest.optuna import OptunaSearch
from ray.tune.suggest.zoopt import ZOOptSearch
from ray.tune.suggest.sigopt import SigOptSearch
from ray.tune.suggest.hebo import HEBOSearch
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.schedulers import HyperBandScheduler
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray import tune
from functools import partial

### Loading hyperparameter configuration file with predefined arguments:

We have a set of predefined configuration files added to CORDS for HPO under cords/configs/HPO/ which can be used directly by loading them as a dotmap object. 

An example of predefined configuration for Hyper-parameter tuning on TREC6 dataset with ASHA as scheduler and TPE as search algorithm can be found below:

```Python
from ray import tune

config = dict(setting= "hyperparamtuning",

# parameter for subset selection
# all settings for subset selection will be fetched from here
subset_config = "configs/SL/config_milo_glove_trec6.py",

# parameters for hyper-parameter tuning
# search space for hyper-parameter tuning
space = dict(learning_rate=tune.uniform(0.001, 0.1), 
        # optimizer= tune.choice(['sgd', 'adam']),
        hidden_size = tune.choice([64, 128, 256]),
        trn_batch_size= tune.choice([16, 32, 64]),
        num_layers = tune.choice([1, 2])
        ),

# tuning algorithm 
search_algo = "TPE",

# number of hyper-parameter set to try
num_evals = 54,

# metric to be optimized, for 'mean_loss' metric mode should be 'min'
metric = "mean_accuracy",
mode = "max",

# scheduler to be used (i.e ASHAScheduler)
# scheduler terminates trials that perform poorly
# learn more here: https://docs.ray.io/en/releases-0.7.1/tune-schedulers.html
# scheduler = 'hyperband',
scheduler = 'ASHA',

# where to store logs
log_dir = "RayLogs/",

# resume hyper-parameter tuning from previous log
# specify 'name' (i.e main_2021-03-09_18-33-56) below
resume = False,

# only required if you want to resume from previous checkpoint
# it can also be specified if you don't want to resume
name = None,

# specify resources to be used per trial
# i.e {'gpu':1, 'cpu':2}
# resources = {'gpu':1, 'cpu':2},
resources = {'gpu':0.5, 'cpu':1},

# if True, trains model on Full dataset with the best parameter selected.
final_train = True,

final_train_type = 'full' # full, gmpb
)
```

Please find a detailed documentation explaining the available configuration parameters in the following readthedocs [page]()

***Loading the predefined configuration file directly using the load_config_data function in CORDS***

In [60]:
from cords.utils.config_utils import load_config_data
param_tuning_cfg = load_config_data('/content/cords/configs/HPO/config_hyper-param_tuning_trec6.py')

In [61]:
from train_sl import TrainClassifier

# Instantiating MILO arguments required for train_sl.py by loading the corresponding configuration file.
Modifying the necessay arguments for single model training

In [62]:
train_cfg_file = '/content/cords/configs/SL/config_milo_glove_trec6.py'
train_cfg = load_config_data(train_cfg_file)
train_cfg.dataset.weight_path = '/content/data/glove.6B/'
train_cfg.model.weight_path = '/content/data/glove.6B/'
train_cfg.dataset.datadir = '/content/data/'
train_cfg.dss_args.global_order_file='/content/data/trec6_all-distilroberta-v1_cossim_disp_min_pc_0.01_global_order.pkl'
train_cfg.dss_args.gc_stochastic_subsets_file = '/content/data/trec6_all-distilroberta-v1_cossim_gc_pc_0.01_0.1_stochastic_subsets.pkl'
"""
Note that we have to do following changes to standard training configuration files
to get them working for Hyper-parameter tuning.
"""
train_cfg.report_tune = True
train_cfg.train_args.print_every = 1

#Instantiating the train classifier class with the loaded train_cfg
train_class = TrainClassifier(train_cfg)

### Get Hyper-parameter search algorithm 

In this example, we will be using Tree-structured parzen estimator(TPE) as the hyper-parameter search algorithm. In the hyper-parameter tuning configuration file, the search algorithm option is given as cfg.search_algo.

In [63]:
method = param_tuning_cfg.search_algo
#Search space
space = param_tuning_cfg.space
#Evaluation metric for configuration evaluation
metric = param_tuning_cfg.metric
#maximum or minimum mode
mode = param_tuning_cfg.mode

"""
Shows all hyper-parameter search algorithm that work with CORDS. We use ray-tune library for hyper-parameter tuning
in CORDS. Hence, all search algorithms given in raytune can be used with CORDS as well.
"""
# HyperOptSearch 
if method == "hyperopt" or method == "TPE":
    search = HyperOptSearch(space, metric, mode)
# BayesOptSearch
elif method == "bayesopt" or method == "BO":
    search = BayesOptSearch(space, metric = metric, mode = mode)
# SkoptSearch
elif method == "skopt" or method == "SKBO":
    search = SkOptSearch(space, metric = metric, mode = mode)
# DragonflySearch
elif method == "dragonfly" or method == "SBO":
    search = DragonflySearch(space, metric = metric, mode = mode)
# AxSearch
elif method == "ax" or method == "BBO":
    search = AxSearch(space, metric = metric, mode = mode)
# TuneBOHB
elif method == "tunebohb" or method == "BOHB":
    search = TuneBOHB(space, metric = metric, mode = mode)
# NevergradSearch
elif method == "nevergrad" or method == "GFO":
    search = NevergradSearch(space, metric = metric, mode = mode)
# OptunaSearch
elif method == "optuna" or method == "OSA":
    search = OptunaSearch(space, metric = metric, mode = mode)
# ZOOptSearch
elif method == "zoopt" or method == "ZOO":
    search = ZOOptSearch(space, metric = metric, mode = mode)
# SigOptSearch
elif method == "sigopt":
    search = SigOptSearch(space, metric = metric, mode = mode)
# HEBOSearch
elif method == "hebo" or method == "HEBO":
    search = HEBOSearch(space, metric = metric, mode = mode)
else:
    search = None


2023-02-08 13:46:42,590	WARNING hyperopt_search.py:192 -- You passed a `space` parameter to <class 'ray.tune.search.hyperopt.hyperopt_search.HyperOptSearch'> that contained unresolved search space definitions. <class 'ray.tune.search.hyperopt.hyperopt_search.HyperOptSearch'> should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `param_space` argument to `tune.Tuner()` instead.


### Get Hyper-parameter scheduler

In this example, we will be using ASHA as the hyper-parameter scheduler algorithm. In the hyper-parameter tuning configuration file, the scheduler option is given as cfg.scheduler.

In [64]:
method = param_tuning_cfg.scheduler

if method == "ASHA" or method == "asha":
    scheduler = AsyncHyperBandScheduler(metric = metric, mode = mode, 
                                        max_t = train_cfg.train_args.num_epochs)
elif method == "hyperband" or method == "HB":
    scheduler = HyperBandScheduler(metric = metric, mode = mode, 
                max_t = train_cfg.train_args.num_epochs)
elif method == "BOHB":
    scheduler = HyperBandForBOHB(metric = metric, mode = mode)
else:
    scheduler = None

scheduler = scheduler

### Utility function that updates the training configuration parameters with new parameters suggested by search algorithm

In [65]:
def update_parameters(config, new_config):
    # a generic function to update parameters
    if 'learning_rate' in new_config:
        config.optimizer.lr = new_config['learning_rate']
    if 'learning_rate1' in new_config:
        config.optimizer.lr1 = new_config['learning_rate1']
    if 'learning_rate2' in new_config:
        config.optimizer.lr2 = new_config['learning_rate2']
    if 'learning_rate3' in new_config:
        config.optimizer.lr3 = new_config['learning_rate3']
    if 'optimizer' in new_config:
        config.optimizer.type = new_config['optimizer']
    if 'nesterov' in new_config:
        config.optimizer.nesterov = new_config['nesterov']
    if 'scheduler' in new_config:
        config.scheduler.type = new_config['scheduler']
    if 'gamma' in new_config:
        config.scheduler.gamma = new_config['gamma']
    if 'epochs' in new_config:
        config.train_args.num_epochs = new_config['epochs']
    if 'trn_batch_size' in new_config:
        config.dataloader.batch_size = new_config['trn_batch_size']
    if 'hidden_size' in new_config:
        config.model.hidden_size = new_config['hidden_size']
    if 'num_layers' in new_config:
        config.model.num_layers = new_config['num_layers']
    return config


### Utility function that takes in the search configuration parameters suggested by hyper-parameter search algorithm, update the training configuration file accordingly, and train the model with the new configuration. 

In [66]:
def param_tune(config, train_cfg=None):
    #update parameters in config dict
    new_config = update_parameters(train_cfg, config)
    train_cfg = new_config
    # turn on reporting to ray every time
    train_cfg.report_tune = True
    train_class.cfg = new_config
    train_class.train()


### Start Hyper-parameter tuning

In [67]:
analysis = tune.run(
          partial(param_tune, train_cfg=train_cfg),
          num_samples=param_tuning_cfg.num_evals,
          search_alg=search,
          scheduler=scheduler,
          resources_per_trial={'gpu': 0.5, 'cpu': 4},
          local_dir=param_tuning_cfg.log_dir+train_cfg.dss_args.type+'/',
          log_to_file=True,
          name=param_tuning_cfg.name,
          resume=param_tuning_cfg.resume)

(func pid=35464) WARNING:datasets.builder:Found cached dataset trec (/root/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2)
100%|██████████| 2/2 [00:00<00:00, 577.89it/s]
(func pid=35658) WARNING:datasets.builder:Found cached dataset trec (/root/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2)
100%|██████████| 2/2 [00:00<00:00, 597.91it/s]


Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,mean_accuracy,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
param_tune_054dafad,2023-02-08_13-51-17,True,,415135ba04bb482aaf57263a5c510a6b,2e6c9a135542,1,0.238532,172.28.0.12,38430,26.6705,26.6705,26.6705,1675864277,0,,1,054dafad,0.0036025
param_tune_067346ab,2023-02-08_13-49-49,True,,2008fa235c3e4c88a08e2d612aa9e96e,2e6c9a135542,1,0.238532,172.28.0.12,37246,26.7673,26.7673,26.7673,1675864189,0,,1,067346ab,0.00383067
param_tune_149d151a,2023-02-08_13-53-10,True,,3c657400486043108499c2e7f00b9805,2e6c9a135542,4,0.442202,172.28.0.12,39867,28.6468,0.40358,28.6468,1675864390,0,,4,149d151a,0.0039618
param_tune_167f2747,2023-02-08_13-50-40,True,,61de95a5a2fd4e3d87af6c429adc66a8,2e6c9a135542,1,0.223853,172.28.0.12,37951,26.5848,26.5848,26.5848,1675864240,0,,1,167f2747,0.00375867
param_tune_239d4109,2023-02-08_13-53-48,True,,b12b676b84fe42c682c00ed830aa002c,2e6c9a135542,1,0.214679,172.28.0.12,40358,28.024,28.024,28.024,1675864428,0,,1,239d4109,0.00381017
param_tune_23fd6dd4,2023-02-08_13-48-21,True,,68aa3fc4474d4b84ae11d714601185e6,2e6c9a135542,1,0.244037,172.28.0.12,36202,28.5301,28.5301,28.5301,1675864101,0,,1,23fd6dd4,0.00403857
param_tune_2e116cc8,2023-02-08_13-50-02,True,,6be536dc3afc43838d6047f6471f09b6,2e6c9a135542,1,0.168807,172.28.0.12,37461,27.1196,27.1196,27.1196,1675864202,0,,1,2e116cc8,0.00369
param_tune_3692baaa,2023-02-08_13-51-55,True,,15b0c9da066e4e74b9187ec35d69c588,2e6c9a135542,1,0.220183,172.28.0.12,38910,27.0981,27.0981,27.0981,1675864315,0,,1,3692baaa,0.00372529
param_tune_4687319b,2023-02-08_13-48-49,True,,845c7027dcdd479ba5e659c9cd2af57f,2e6c9a135542,1,0.223853,172.28.0.12,36482,28.0294,28.0294,28.0294,1675864129,0,,1,4687319b,0.00376701
param_tune_65509966,2023-02-08_13-49-25,True,,7a5a2faca0da4a458d57c322754ae0b9,2e6c9a135542,1,0.222018,172.28.0.12,36970,26.5046,26.5046,26.5046,1675864165,0,,1,65509966,0.00374651


(func pid=35464) Epoch: 2, requires subset selection. 
(func pid=35464) Epoch: 3, requires subset selection. 
(func pid=35464) Epoch: 2, requires subset selection. 
(func pid=35464) Epoch: 3, requires subset selection. 
(func pid=35464) Epoch: 4, requires subset selection. 
(func pid=35464) Epoch: 5, requires subset selection. 
(func pid=35464) Epoch: 6, requires subset selection. 
(func pid=35464) Epoch: 7, requires subset selection. 
(func pid=35464) Epoch: 8, requires subset selection. 
(func pid=35464) Epoch: 9, requires subset selection. 
(func pid=35464) Epoch: 10, requires subset selection. 
(func pid=35464) Epoch: 11, requires subset selection. 
(func pid=35464) Epoch: 12, requires subset selection. 
(func pid=35464) Epoch: 13, requires subset selection. 
(func pid=35464) Epoch: 14, requires subset selection. 
(func pid=35464) Epoch: 15, requires subset selection. 


(func pid=35999) WARNING:datasets.builder:Found cached dataset trec (/root/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2)
100%|██████████| 2/2 [00:00<00:00, 634.30it/s]
(func pid=36202) WARNING:datasets.builder:Found cached dataset trec (/root/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2)
100%|██████████| 2/2 [00:00<00:00, 602.15it/s]
(func pid=36482) WARNING:datasets.builder:Found cached dataset trec (/root/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2)
100%|██████████| 2/2 [00:00<00:00, 584.98it/s]
(func pid=36685) WARNING:datasets.builder:Found cached dataset trec (/root/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2)
100%|██████████| 2/2 [00:00<00:00, 598.20it/s]


(func pid=36685) Epoch: 2, requires subset selection. 


...

(func pid=41339) WARNING:datasets.builder:Found cached dataset trec (/root/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2)
100%|██████████| 2/2 [00:00<00:00, 549.78it/s]


(func pid=41107) Epoch: 2, requires subset selection. 
(func pid=41107) Epoch: 3, requires subset selection. 


(func pid=41611) WARNING:datasets.builder:Found cached dataset trec (/root/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2)
100%|██████████| 2/2 [00:00<00:00, 548.35it/s]


(func pid=41339) Epoch: 2, requires subset selection. 
(func pid=41339) Epoch: 3, requires subset selection. 


(func pid=41841) WARNING:datasets.builder:Found cached dataset trec (/root/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2)
100%|██████████| 2/2 [00:00<00:00, 621.65it/s]
2023-02-08 13:55:44,927	INFO tune.py:762 -- Total run time: 538.29 seconds (538.16 seconds for the tuning loop).


### Get best hyper-parameter configuration

In [68]:
best_config = analysis.get_best_config(metric="mean_accuracy", mode="max")
print("Best Config: ", best_config)

Best Config:  {'hidden_size': 256, 'learning_rate': 0.008139721964940762, 'num_layers': 1, 'trn_batch_size': 16}
